In [29]:
import spotipy
import spotipy.util as util
import pickle
import sys
import requests
from tqdm import tqdm
import os
import pandas as pd
import numpy as np
from multiprocessing import Pool
from fuzzywuzzy import fuzz
import string
import json
import unidecode
import sys
!{sys.executable} -m pip install pytrends
import time
from datetime import datetime
from datetime import date
from dateutil.relativedelta import relativedelta
from pytrends.request import TrendReq


In [52]:
#Read song data frame from pkl file
with open('song_df_100.pkl', 'rb') as f:
        song_df = pickle.load(f)

In [53]:
song_df

,artist,title,isnew,rank,date,spotify_id,spotify_explicit,spotify_duration_ms,spotify_disc_number,spotify_track_number,...,spotify_acousticness,spotify_instrumentalness,spotify_liveness,spotify_valence,spotify_tempo,spotify_time_signature,spotify_album_id,spotify_album_release_date,spotify_album_release_date_precision,album_label
0,Lewis Capaldi,Someone You Loved,False,1,2019-11-16,7qEHsqek33rTcFNT9PFqLf,False,182160.0,1.0,4.0,...,0.7510,0.00000,0.1050,0.4460,109.891,4.0,5658aM19fA3JVwTK6eQX70,2019-05-17,day,Vertigo Berlin
1,Post Malone,Circles,False,2,2019-11-16,21jGcNKet2qwijlDFuPiPb,False,215280.0,1.0,6.0,...,0.1920,0.00244,0.0863,0.5530,120.042,4.0,4g1ZRSobMefqF6nelkgibi,2019-09-06,day,Republic Records
2,Shawn Mendes & Camila Cabello,Senorita,False,3,2019-11-16,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN
3,Lizzo,Truth Hurts,False,4,2019-11-16,5qmq61DAAOUaW8AUo8xKhh,True,173325.0,1.0,13.0,...,0.1100,0.00000,0.1230,0.4120,158.087,4.0,74gSdSHe71q7urGWMMn3qB,2019-05-03,day,Nice Life/Atlantic
4,Selena Gomez,Lose You To Love Me,False,5,2019-11-16,1HfMVBKM75vxSfsQ5VefZ5,False,206458.0,1.0,1.0,...,0.5760,0.00000,0.2100,0.0916,101.993,4.0,3tBkjgxDqAwss76O1YHsSY,2019-10-23,day,Interscope Records
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113178,Spice Girls,Spice Up Your Life,False,79,1998-03-21,5qGwqO0lkbBXw4xNfzT7SF,False,173666.0,1.0,1.0,...,0.0105,0.03410,0.4030,0.7020,124.016,4.0,3sr6lAuO3nmB1u8ZuQgpiX,1997-01-01,day,Virgin Records
113192,Phajja,"So Long (Well, Well, Well)",False,93,1998-03-21,7phS4S15qLjPC7JlFnzWql,False,262333.0,1.0,23.0,...,0.4320,0.00168,0.0446,0.5450,79.913,4.0,6qbvgcaWgMn2BQY6FhdOm2,2018-11-16,day,Warner Music Group - X5 Music Group
113194,Sting & The Police,Roxanne `97 - Puff Daddy Remix,False,95,1998-03-21,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN
113198,Shania Twain,Don't Be Stupid (You Know I Love You),False,99,1998-03-21,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN


In [66]:
#Drop lines containing N/A
song_dropna = song_df.dropna().copy()
song_dropna

0         2019-05-17
1         2019-09-06
3         2019-05-03
4         2019-10-23
5         2016-03-09
             ...    
113099    1997-09-29
113147          1997
113178    1997-01-01
113192    2018-11-16
113199    2000-01-01
Name: spotify_album_release_date, Length: 4969, dtype: object


In [61]:
from pytrends.request import TrendReq
from random import randint

In [82]:
#this function returns the mean trend of the past n months of a song in song_df 
def google_trend(song_df, song, n):
    trend_date = song_df.loc[song,"spotify_album_release_date"]
    dt = datetime.strptime(trend_date, '%Y-%m-%d')
    
    dt_start = dt + relativedelta(months=-n)
   
    trend_artist = song_df.loc[song,"artist"]
    kw_list = [trend_artist]
    
    #set dataframe, the earliset is 1/1/2014
    earliest_date = '2008-3-12'
    google_earliest = datetime.strptime(earliest_date, '%Y-%m-%d')
    if dt_start >= google_earliest:
        pytrends = TrendReq(hl='en-US', tz=360)
        time_period = dt_start.strftime("%Y") + '-' + dt_start.strftime("%m") + '-' + dt_start.strftime("%d") + ' ' + dt.strftime("%Y") + '-' + dt.strftime("%m") + '-' + dt.strftime("%d")
        pytrends.build_payload(kw_list, cat=0, timeframe= time_period, geo='', gprop='')
        trend = pytrends.interest_over_time()
        if not trend.empty:
            trend_mean=np.mean(trend.iloc[:,0])
            return trend_mean
        else:
            return 0
    elif dt_start < google_earlest and dt > google_earliest:
        pytrends = TrendReq(hl='en-US', tz=360)
        time_period = google_earliest.strftime("%Y") + '-' + google_earliest.strftime("%m") + '-' + dt_startgoogle_earliest.strftime("%d") + ' ' + dt.strftime("%Y") + '-' + dt.strftime("%m") + '-' + dt.strftime("%d")
        pytrends.build_payload(kw_list, cat=0, timeframe= time_period, geo='', gprop='')
        trend = pytrends.interest_over_time()
        if not trend.empty:
            trend_mean=np.mean(trend.iloc[:,0])
            return trend_mean
        else:
            return 0
    elif dt <= google_earliest:
        trend_mean = -1
        return trend_mean
        
print(google_trend(song_df, 0, 3))

print(len(song_df.index))

36.43333333333333
8519


In [83]:
#add the google trend to song_dropna as a new feature

def trend_feature(song_dropna, m, n):
    i=m
    for song in song_dropna[song_dropna["spotify_album_release_date"].notnull()].index:
        if i>=m and i<n:
            try: 
                trend_mean = google_trend(song_dropna, song, 3)
            except requests.exceptions.Timeout:
                print("Timeout occurred")

            song_dropna.loc[song, 'google trend'] = trend_mean
            i=i+1

            print("Row No. ")
            print(song)
            print("Mean trend: ")
            print(song_dropna.loc[song, "google trend"])
            

        time.sleep(randint(3,10))

In [85]:
len(song_dropna.index)

4969

In [86]:
#divide in to groups of 1000 and acquire the data

trend_feature(song_dropna, 0, 1000)
#trend_feature(song_dropna, 1000, 2000)
#trend_feature(song_dropna, 2000, 3000)
#trend_feature(song_dropna, 3000, 4000)
#trend_feature(song_dropna, 4000, len(song_dropna.index))



Row No. 
0
Mean trend: 
36.43333333333333
Row No. 
1
Mean trend: 
24.849462365591396
Row No. 
3
Mean trend: 
23.522222222222222
Row No. 
4
Mean trend: 
16.806451612903224
Row No. 
5
Mean trend: 
15.08695652173913
Row No. 
7
Mean trend: 
30.64516129032258
Row No. 
8
Mean trend: 
42.81720430107527
Row No. 
11
Mean trend: 
23.945054945054945
Row No. 
13
Mean trend: 
23.580645161290324
Row No. 
15
Mean trend: 
25.376344086021504
Row No. 
18
Mean trend: 
28.0752688172043
Row No. 
22
Mean trend: 
32.01075268817204
Row No. 
23
Mean trend: 
77.19354838709677
Row No. 
24
Mean trend: 
3.076923076923077
Row No. 
26
Mean trend: 
28.76086956521739
Row No. 
27
Mean trend: 
38.225806451612904
Row No. 
28
Mean trend: 
8.129032258064516
Row No. 
32
Mean trend: 
24.677419354838708
Row No. 
33
Mean trend: 
29.397849462365592
Row No. 
36
Mean trend: 
12.258064516129032
Row No. 
37
Mean trend: 
23.703296703296704
Row No. 
40
Mean trend: 
31.82608695652174
Row No. 
41
Mean trend: 
52.54838709677419
Row No. 

ConnectionError: HTTPSConnectionPool(host='trends.google.com', port=443): Max retries exceeded with url: /?geo=US (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f8cd776fdd8>: Failed to establish a new connection: [Errno 111] Connection refused',))